# Memory / Chat

Wir haben schon das Chat-Modell von OpenAI genutzt. Allerdings haben wir das noch nicht wirklich als Chat benutzt, da die bisherigen Aufgaben stets nur ein Aufgaben + Antwortpaar produziert haben.
Wichtig zu wissen ist, dass die ganzen Sprachmodelle, die wir für Chat-Bots nutzen, keine Daten sich behalten. D.h. der Chatverlauf existiert dort nicht. Wenn man also eine Frage stellt, die sich auf vorherige Daten aus dem Verlauf beziehen, dann weiß die KI nichts mehr darüber.
Damit das Ganze trotzdem möglich ist, muss man stets die gesamte (oder Teile) der Konversation bei jedem neuen Prompt mitschicken. Der Chatverlauf wird dadurch zum Teil des Kontext und die KI hat ein Kurzzeitgedächtnis bekommen.

Erstmal ein Beispiel, wie man das zu Fuß macht. Danach schauen wir uns an, wie man das mit LangChain automatisieren kann.

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.chat import SystemMessage, HumanMessage, AIMessage

messages = [
    SystemMessage(content="Du bist ein hilfreicher KI Assistent, der mir bei meinen Fragen und Aufgaben hilft"),
    ]

llm = ChatOpenAI()

user_prompt = "Erstelle bitte eine Liste der fünf größten Sprachmodelle" # Hier könnten wir auch tatsächlich eine Eingabeaufforderung starten. Zum Zeigen gebe ich aber den Text vor
messages.append(HumanMessage(content=user_prompt))

response = llm(messages)
messages.append(response)
print(response)

messages.append(HumanMessage(content="Welche stammen von OpenAI?"))

response = llm(messages)
messages.append(response)
print(response)

Erstmal sehen wir, dass wir unterschiedliche Message-Typen haben: SystemMessage, Humanmessage und AIMessage. Diese widerspiegeln die verschiedenen Rollen, die von ChatGPT erwartet werden.
Der Ablauf ist stets so, dass wir der Liste der Nachrichten die Fragen und Antworten hinzufügen. Die _messages_ Liste ist somit unser Chatverlauf und praktisch unser Speicher.

Damit wir das nicht manuell machen müssen, bietet LangChain _Memory_ an. Davon gibt es unterschiedliche Implementierungen. Der einfachste Speicher merkt sich ganz einfach jede Nachricht, so wie wir es oben gemacht haben. Allerdings ist der Platz im Kontext begrenzt und ein Chat, der länger geführt wird, könnte dazu führen, dass die maximale Kontextgröße überschritten wird. Dafür gibt es Speicher, die entweder nur die letzten n Nachrichten behält oder noch ausgefeilter, ein Speicher, der die bisherige Kommunikation zusammenfasst, um Platz zu sparen.

Schauen wir uns das Beispiel von oben mit LangChain Memory an:

In [ ]:
from langchain.memory import ChatMessageHistory

llm = ChatOpenAI()

history = ChatMessageHistory()

history.add_message(SystemMessage(content="Du bist ein hilfreicher KI Assistent, der mir bei meinen Fragen und Aufgaben hilft"))

history.add_user_message("Erstelle bitte eine Liste der fünf größten Sprachmodelle")

response = llm(history.messages)

history.add_ai_message(response.content)

print(response)

Das Beispiel soll zeigen, wie die Grundstrukturen eines Speichers aussehen. Im Grunde genommen ist das mit den manuellen Schritten erstmal fast identisch und hilft uns erstmal nicht das Ganze zu automatisieren.
In Kombination mit einer _vernünftigen_ Speicher-Implementierung und einer Chain sieht das Ganze wesentlich besser aus:

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain

memory = ConversationBufferMemory()

llm = ChatOpenAI()

conversation = ConversationChain(
    llm=llm, 
    memory=memory
)

response = conversation.predict(input="Welche Bedeutung hat die Angabe der Anzahl der Parameter bei einem Sprachmodell?")

print(response)

In [ ]:
response = conversation.predict(input="Wie viele sind es bei GPT-3?")

print(response)

Wir sehen also, dass die Chain sich automatisch darum kümmert die Nachrichten in den dazugehörigen Speicher zu laden. Durch die Verwendung von _ConversationChain_ haben wir automatisch ein Prompttemplate, das einen _input_ Parameter erwartet, den wir an die _predict_ Funktion übergeben.

In [ ]:
memory

Die Ausgabe der _memory_ Variable zeigt, wie die ganze Konversation mitgeschnitten wurde.

Jetzt sind wir praktisch nur eine Schleife weg von einem Chatbot:

In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

memory = ConversationBufferMemory()

llm = ChatOpenAI()

conversation = ConversationChain(
    llm=llm, 
    memory=memory
)

while True:
    prompt = input("Prompt:")
    response = conversation.predict(input=prompt)
    print(response)

Einmal hübsch machen, Weboberfläche, fertig ist der Chatbot!

Jetzt kann es sein, dass wenn wir viel Eingeben (und ausgeben lassen), das irgendwann der Kontext zu voll wird. Wie oben erwähnt können wir dafür verschiedene Speicherarten nutzen.

Dieser hier begrenzt den Chat auf ein bestimmtes Token-Limit (Token: Sprachmodelle arbeiten mit Tokens anstelle von Wörtern. Manche Wörter, insbesondere in Deutsch, bestehen aus mehreren Tokens.)

In [9]:
from langchain.memory import ConversationTokenBufferMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()


# Wir übergeben das Sprachmodell + Maximale Anzahl an Tokens. Das Sprachmodell dient dazu die Anzahl der Tokens zu ermitteln
# Hier wir zu Demonstrationszwecken sehr kleines Limit angegeben
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=100)

conversation = ConversationChain(
    llm=llm, 
    memory=memory
)

response = conversation.predict(input="Hallo!")
print(f"Antwort: {response}")
print(f"Der Inhalt des Speichers jetzt:\n{memory.buffer_as_str}\n\n")
    
response = conversation.predict(input="Erstelle mir bitte eine Liste der letzten 10 Präsidenten der USA")
print(f"Antwort: {response}")
print(f"Der Inhalt des Speichers jetzt:\n{memory.buffer_as_str}\n\n")

response = conversation.predict(input="Welcher davon ist der aktuelle Präsident?")
print(f"Antwort: {response}")
print(f"Der Inhalt des Speichers jetzt:\n{memory.buffer_as_str}\n\n")

Hello! How can I assist you today?
Der Inhalt des Speichers jetzt:
 Human: Hallo!
AI: Hello! How can I assist you today?


Natürlich! Hier ist eine Liste der letzten 10 Präsidenten der USA:

1. Joe Biden (2021-heute)
2. Donald Trump (2017-2021)
3. Barack Obama (2009-2017)
4. George W. Bush (2001-2009)
5. Bill Clinton (1993-2001)
6. George H.W. Bush (1989-1993)
7. Ronald Reagan (1981-1989)
8. Jimmy Carter (1977-1981)
9. Gerald Ford (1974-1977)
10. Richard Nixon (1969-1974)

Das sind die letzten 10 Präsidenten der USA.
Der Inhalt des Speichers jetzt:
 


Der aktuelle Präsident der Vereinigten Staaten ist Joe Biden. Er wurde am 20. Januar 2021 als 46. Präsident der USA vereidigt.
Der Inhalt des Speichers jetzt: Human: Welcher davon ist der aktuelle Präsident?
AI: Der aktuelle Präsident der Vereinigten Staaten ist Joe Biden. Er wurde am 20. Januar 2021 als 46. Präsident der USA vereidigt.




Zu guter letzt ein schauen wir uns einen etwas ausgefeilteren Speicher an. Dieser löscht nicht einfach alte Konversationen, sondern fasst den bisherigen Verlauf zusammen.
Zu beachten ist, dass das zusätzliche Aufrufe zu unserem Sprachmodell bedeuten und somit auch weitere Kosten verursachen kann!

In [12]:
from langchain.memory import ConversationSummaryMemory
from langchain.chains import ConversationChain
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI()


# Wir übergeben das Sprachmodell. Das Sprachmodell dient dazu die Zusammenfassung zu erstellen.
memory = ConversationSummaryMemory(llm=llm)

conversation = ConversationChain(
    llm=llm, 
    memory=memory,
    verbose=True
)

response = conversation.predict(input="Hallo!")
print(f"Antwort: {response}")
    
response = conversation.predict(input="Erstelle mir bitte eine Liste der letzten 10 Präsidenten der USA")
print(f"Antwort: {response}")

response = conversation.predict(input="Welcher davon ist der aktuelle Präsident?")
print(f"Antwort: {response}")




> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hallo!
AI:



> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The human greets the AI and the AI asks how it can assist the human.
Human: Erstelle mir bitte eine Liste der letzten 10 Präsidenten der USA
AI:

> Finished chain.


> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
The AI asks how it can assist the human. The human requests a list of the last 10 presidents of the USA. The AI provides the requested list and asks if there is anything e